Copyright © 2023, SAS Institute Inc., Cary, NC, USA.  All Rights Reserved.
SPDX-License-Identifier: Apache-2.0

# Model Migration: Moving Models from SAS Viya 3.5 to SAS Viya 4 in SAS Model Manager

This notebook provides an example scenario for downloading a model from SAS Viya 3.5, converting it in to a format acceptable for SAS Viya 4, then uploads the model to a SAS Viya 4 server. In order to complete the conversion, the following files are modified:
- dmcas_epscorecode.sas (deleted)
- dmcas_packagescore.sas (deleted)
- score.sas (deleted)
- ModelProperties.json
- fileMetadata.json
- python score code file

Over the course of this notebook, a model from SAS Viya 3.5 is downloaded to a specified directory. The model contents are modified to the format expected in SAS Viya 4, then rezipped into a single model zip file. This new model is then uploaded to a SAS Viya 4 server. Optionally, the model zip file can be deleted, leaving the directory in an empty state for the next model to be migrated.

In [ ]:
import io
import zipfile
from pathlib import Path

from sasctl._services.model_repository import ModelRepository as mr
from sasctl.core import is_uuid
from sasctl.utils import convert_model_zip
from sasctl import Session

In [ ]:
def set_model_directory(path=Path.cwd()):
    """
    Set or create an empty directory path for model migration

    Parameters
    ----------
    path : str or Path, optional
        A string or Path object pointing at an empty or nonexistent directory; default is the current working directory.

    Returns
    -------
    dir_path : Path
        A Path object indicating the empty directory for models.
    Raises
    ------
    FileExistsError:
        The provided directory is not empty.
    """
    # Check if provided path is a valid directory and is empty
    if Path(path).is_dir():
        # Check if directory is not empty
        if any(Path(path).iterdir()):
            raise FileExistsError(f"The directory {str(Path(path))} is not empty. Please either provide" +
                                  " an empty directory path or path of a directory to be created.")
        return path
    elif Path(path).is_file():
        print("The provided path points at a file. Attempting to create a new directory with the name of the file.")
        new_path = set_model_directory(Path(path).parent / Path(path).stem)
        return new_path
    else:
        Path(path).mkdir(parents=True)
        print(f"The {str(Path(path))} directory was created.")
        return path

In [ ]:
def download_viya35_model(path, model35, project35=None):
    """ Download the model zip for a SAS Viya 3.5 model from SAS Model Manager. Then unzip the model and delete the
    zip archive downloaded.

    Parameters
    ----------
    path : Path object
        Parent directory for models to be migrated.
    model : str or dict
        The name or id of the model, or a dictionary representation of the model.
    project : str or dict, optional
        The name or id of the project, or a dictionary representation of the project. Default value is None.

    Returns
    -------
    model : dict
        A dictionary representation of the model.
    project : dict or None
        A dictionary representation of the project.
    """
    if project35:
        project35 = mr.get_project(project35)
        if is_uuid(model35):
            model35 = mr.get_model(model35)
        elif isinstance(model35, dict) and "id" in model35:
            model35 = model35
        else:
            model35 = mr.list_models(filter=f"and(eq(projectName,'{project35.name}'),eq(name,'{model35}'))")[0]
    else:
        # If only a name and no project is provided, the correct model may not be found from the repository
        if (not is_uuid(model35)) and (not isinstance(model35, dict)):
            print("No project was provided for model {}.".format(model) + " If another model with the same name" +
                  " exists in the repository, it is not guaranteed to be the model you are attempting to reference.")
        model35 = mr.get_model(model35)

    model_zip = mr.get(f"models/{model35.id}", params={"format": "zip"}, format_="content")
    zip_path = Path(path) / (model35.name + ".zip")
    with open(zip_path, "wb") as z_file:
        z_file.write(model_zip)
    print(f"Model {model35.name} downloaded to {str(zip_path)}.")
    with zipfile.ZipFile(str(zip_path), "r") as zip_contents:
        zip_contents.extractall(path)
    print("Model contents have been extracted from {}.".format(str(zip_path)))
    zip_path.unlink()
    try:
        return model35, mr.get_project(model35.projectId)
    except AttributeError:
        return model35, None

In [ ]:
def convert_and_upload_model(path, model4, project4=None):
    """ Convert the model Viya 3.5 model contents to the Viya 4 model content format, then upload the model to a
    specified Viya 4 server.

    Parameters
    ----------
    path : str or Path object
        A string or Path object where the Viya 3.5 model contents are located.
    model : str or dict
        The name or id of the model, or a dictionary representation of the model.
    project : str or dict, optional
        The name or id of the project, or a dictionary representation of the project. Default value is None.
    """
    convert_model_zip(path)
    
    project4 = mr.get_project(project4)
    # If no project is found, create a new one
    if not project4:
        project4 = mr.create_project(project4)

    # List all files in directory
    files = [x for x in path.glob("**/*") if x.is_file()]
    # Zip all files in directory into a zip archive
    with zipfile.ZipFile(str(path / (model4 + ".zip")), "w") as zip_file:
        for file in files:
            zip_file.write(str(file), arcname=file.name)
            file.unlink()

    # Import the model into SAS Model Manager
    with open(path / (model4 + ".zip"), "rb") as zip_file:
        if project4:
            response = mr.import_model_from_zip(model4, project4, io.BytesIO(zip_file.read()))
        else:
            params = {"name": model4.name, "type": "ZIP", "versionOption": "latest"}
            params = "&".join("{}={}".format(k, v) for k, v in params.items())
            response = mr.post(
                "/models#octetStream",
                data=io.BytesIO(zip_file.read()).read(),
                params=params,
                headers={"Content-Type": "application/octet-stream"},
            )
    # Uncomment if you want to delete the converted model zip
    # Path(path / model4.name + ".zip").unlink()

    try:
        print(
            "Model was successfully imported into SAS Model Manager as {} with UUID: {}.".format(
                response.name, response.id
            )
        )
    except AttributeError:
        print("Model, {}, failed to import into SAS Model Manager.".format(model.name))

In [ ]:
model_name = "ExampleModel"
project_name = "ExampleProject"
viya35_host = "demo3.5.sas.com"
viya4_host = "demo4.sas.com"
username = "sasuser"
password = "saspass"
path = Path.cwd() / "data/ModelMigration"

In [ ]:
set_model_directory(path)

In [ ]:
with Session(viya35_host, username, password, protocol="http"):
    model, project = download_viya35_model(path, model_name, project_name)
with Session(viya4_host, username, password, protocol="http"):
    convert_and_upload_model(path, model_name, project_name)